<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/gan/c1_w3_wgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wasserstein GAN with Gradient Penalty (WGAN-GP)

## Motivation

Training GANs is hard. Models may never converge and mode collapses are common. To move forward, we need to make incremental improvements or embrace optimal cost functions.

### WGAN-GP improvements

WGAN-GP enhances training stability. As shown below, when the model design is less optimal, WGAN-GP can still create good results while the original GAN cost function fails.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/images/wgan-gp-1.png" width=600>


Below is the inception score using different methods. The experiment from WGAN-GP paper demostrates better image quality and convergence comparing with WGAN. However, DCGAN demostrates slightly better image quality and it converges faster. But the inception score for WGAN-GP is more stable when it starts converting.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/images/wgan-gp-2.png" width=500>

> ### So what is the benefit of WGAN-GP if it cannot beat DCGAN?
> - `The major advantage of WGAN-GP is its convergency`. It makes training more stable and therefore easier to train. As WGAN-GP helps model to converge better, we can use a more complex model lie deep ResNet for generation and the discriminator.  
> - When using the Wasserstein loss function, we should train the critic to `converge to ensure that the gradients for the generator are accurate`. This is `in contrast to a standard GAN`, where it is important `not to let the discrimininator get too strong, to avoid vanishing gradients`. Helping to reduce one of the main difficulties of training GANs - How to balance the training of the discriminator and generator.

## Wasserstein Distance

Wasserstein GAN (WGAN) proposes a new cost function using Wasserstein distance that has a smoother gradient everyware. WGAN learns regarless of the generator performance. The diagram below repeats a similar plot on the value $D(X)$ fro both GAN and WGAN. For GAN (the red line), it fills with areas with diminishing or exploiding gradient. For WANG (the blue line), the gradient is smoother everywhere and learns better even when the generator is not producing good images.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/gan/images/wgan-4.png" width=300>



GAN `discriminator loss` minimization $$min_D−(\mathbb{E}_{x∼pX}[log\,D(x)] + \mathbb{E}_{z∼pZ}[log\,(1−D(G(z)))])$$

To train the GAN `generator` G, we calculate the loss when comparing predictions for generated images $p_i=D(G(zi))$ to the response y_i=1. Therefore for the GAN generator, minimizing the loss function can be written as follows:

GAN `generator loss` minimization:
$$min_G−(\mathbb{E}_{z∼pZ}[log\,D(G(z))])$$

### Wasserstein loss function

First, the Wasserstein loss requires that we use $y_i=1$ and $y_i=-1$ as labels, rather than 1 and 0. We also remove the sigmoid activation from the final layer of the **discriminator(critic)**, so that predictions $p_i$ are no longer constrained to fall in the range $[0,1]$, but instead can now be any number in the range $[–∞, ∞]$. For this reason, the discriminator in a WGAN is usually referred to as a critic.The Wasserstein loss function is then defined as follows:

Wasserstein loss:
$$-\frac{1}{n}\sum_{i=1}^{n}{(y_i\,p_i)}$$

To train the WGAN critic $D$, we calculate the loss when comparing predictions for a real images $p_i=D(x_i)$ to the response $y_i=1$ and predictions for generated images $p_i=D(G(z_i))$ to the response $y_i=-1$. Therefore for the WGAN critic, minimizing the loss function can be written as follows:

WGAN critic loss minimization:

$$min_D−(\mathbb{E}_{x∼pX}[D(x)]−\mathbb{E}_{z∼pZ}[D(G(z))])$$

### The Lipschitz Constraint

It may surprise you that we are now allowing the critic to output any number in the range $[–∞, ∞]$, rather than applying a sigmoid function to restrict the output to the usual $[0, 1]$ range. The Wasserstein loss can therefore be very large, which is unsettling—usually, large numbers in neural networks are to be avoided!

In fact, the authors of the WGAN paper show that for the Wasserstein loss function to work, we also need to place an additional constraint on the critic. Specifically, it is required that the critic is a `1-Lipschitz continuous` function. Let’s pick this apart to understand what it means in more detail.

The critic is a function $D$ that converts an image into a prediction. We say that this function is 1-Lipschitz if it satisfies the following inequality for any two input images x1 and x2:

$$\frac{|D(x_1)−D(x_2)|}{|x_1−x_2|}\leq1$$

Here, $x_1 – x_2$ is the average pixelwise absolute difference between two images and $|D(x_1)−D(x_2)||D(x_1)-D(x_2)|$ is the absolute difference between the critic predictions. Essentially, we require a limit on the rate at which the predictions of the critic can change between two images (i.e., the absolute value of the gradient must be at most 1 everywhere).

## Key differences between GANs and WGANs

- A WGAN uses the Wasserstein loss.

- The WGAN is trained using labels of `1 for real` and `–1 for fake`.

- There is `no need for the sigmoid activation` in the final layer of the WGAN critic.

- `Clip the weights` of the critic after each update.

- Train the critic multiple times for each update of the generator.

### Goals
In this notebook, you're going to build a Wasserstein GAN with Gradient Penalty (WGAN-GP) that solves some of the stability issues with the GANs that you have been using up until this point. Specifically, you'll use a special kind of loss function known as the W-loss, where W stands for Wasserstein, and gradient penalties to prevent mode collapse.

*Fun Fact: Wasserstein is named after a mathematician at Penn State, Leonid Vaseršteĭn. You'll see it abbreviated to W (e.g. WGAN, W-loss, W-distance).*

### Learning Objectives
1.   Get hands-on experience building a more stable GAN: Wasserstein GAN with Gradient Penalty (WGAN-GP).
2.   Train the more advanced WGAN-GP model.

## Generator and Critic

You will begin by importing some useful packages, defining visualization functions, building the generator, and building the critic. Since the changes for WGAN-GP are done to the loss function during training, you can simply reuse your previous GAN code for the generator and critic class. Remember that in WGAN-GP, you no longer use a discriminator that classifies fake and real as 0 and 1 but rather a critic that scores images with real numbers.

In [3]:
import torch
import matplotlib.pyplot as plt

from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader

torch.manual_seed(0)
torch.__version__

'1.7.0+cu101'

In [4]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

def make_grad_hook():
    '''
    Function to keep track of gradients for visualization purposes, 
    which fills the grads list when using model.apply(grad_hook).
    '''
    grads = []
    def grad_hook(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            grads.append(m.weight.grad)
    return grads, grad_hook

### Generator and Noise

In [ ]:
def get_noise(n_samples, z_dim, device='cpu'):
  

In [7]:
class Generator(nn.Module):
  '''
  Generator Class
  Values:
      z_dim: the dimension of the noise vector, a scalar
      im_chan: the number of channels in the images, fitted for the dataset used, a scalar
            (MNIST is black-and-white, so 1 channel is your default)
      hidden_dim: the inner dimension, a scalar
  '''

  def __init__(self, z_dim=10, in_chan=1, hidden_dim=64):
    super(Generator, self).__init__()
    self.z_dim = z_dim
    # build the neural network
    self.gen = nn.Sequential(
      self.make_gen_block(z_dim, hidden_dim * 4),
      self.make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size=4, 
                          stride=1),
      self.make_gen_block(hidden_dim * 2, im_chan, kernel_size=4, 
                          final_layer=True)
    )
  
  def make_gen_block(self, input_channels, output_channels, kernel_size=3, 
                     stride=2, final_layer=False):
    '''
    Function to return a sequence of operations corresponding to a generator block of DCGAN;
    a transposed convolution, a batchnorm (except in the final layer), and an activation.
    Parameters:
        input_channels: how many channels the input feature representation has
        output_channels: how many channels the output feature representation should have
        kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
        stride: the stride of the convolution
        final_layer: a boolean, true if it is the final layer and false otherwise 
                  (affects activation and batchnorm)
    '''
    if not final_layer:
      return nn.Sequential(
          nn.ConvTranspose2d(input_channels, output_channels, kernel_size, 
                             stride),
          nn.BatchNorm2d(output_channels),
          nn.ReLU(inplace=True),
      )
    else:
      return nn.Sequential(
          nn.ConvTranspose2d(input_channels, output_channels, kernel_size, 
                             stride),
          nn.Tanh(),
      )

  def forward(self, noise):
    '''
    Function for completing a forward pass of the generator: Given a noise tensor,
    returns generated images.
    Parameters:
        noise: a noise tensor with dimensions (n_samples, z_dim)
    '''
    x = noise.view(len(noise), self.z_dim, 1, 1)